In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import math
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import GridSearchCV

#if this doesn't run then do 'pip install causalinference' in command line
from causalinference import CausalModel

#if this doesn't run then do 'pip install rpy2' in command line
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.vectors import StrVector
from rpy2.robjects import FloatVector, Formula

In [2]:
lowDim_dataset = pd.read_csv('../data/lowDim_dataset.csv')
highDim_dataset = pd.read_csv('../data/highDim_dataset.csv')

## A1) Propensity Score Full Matching

### 1. Mahalanobis Metric (Does not need propensity score)

Mahalanobis distance is 
$$D_{ij} = (X_i-X_j)^T\Sigma^{-1}(X_i-X_j)$$
where $\Sigma$ is the covariance matrix of $X$ in the pooled treatment and full control groups.

In [3]:
X=lowDim_dataset.iloc[:,2:].values
A=lowDim_dataset['A'].values
Y=lowDim_dataset['Y'].values

In [4]:
#don't need this anymore
#dist_matrix_mahalanobis = pairwise_distances(X,metric='mahalanobis')

#full matching:
#fullmatch(match_on(A~X,data=lowDim_dataset,method='mahalanobis'),data=df)

### 2. Propensity Score

In [5]:
#creating gbm model for calculating propensity score

gbm = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 2, min_samples_leaf = 1,
                                min_samples_split = 2, n_estimators = 150).fit(X,A)

propensity_scores = [x[1] for x in gbm.predict_proba(X)]

In [6]:
lowDim_dataset_propensity = lowDim_dataset.copy(deep=True)
lowDim_dataset_propensity['propensity_score'] = propensity_scores

#full matching:
#matchit(A~propensity_scores,data=lowDim_dataset_propensity,method='full')
#OR
#fullmatch(match_on(A~propensity_scores,data=lowDim_dataset_propensity,method='euclidean'),data=lowDim_dataset_propensity,method='full')

### 3. Linear Propensity Score

In [7]:
def logit(x):
    return math.log(x/(1-x))

In [8]:
#negative values make sense since if you look at a graph of log(x/(1-x))
linear_propensity_scores = [logit(x) for x in propensity_scores]

In [9]:
lowDim_dataset_linear_propensity = lowDim_dataset.copy(deep=True)
lowDim_dataset_linear_propensity['linear_propensity_score'] = linear_propensity_scores

#full matching:
#matchit(A~linear_propensity_scores,data=lowDim_dataset_linear_propensity,method='full')
#OR
##fullmatch(match_on(A~linear_propensity_scores,data=lowDim_dataset_linear_propensity,method='euclidean'),data=lowDim_dataset_linear_propensity,method='full')

## Grid Search 

In [10]:
#low dim grid search (commented out since it takes a few minutes to run)

#params = {'learning_rate':[0.01,0.05,0.1,0.5], 'max_depth': [1,2,3,4], 'n_estimators':[50,100,150],
#          'min_samples_leaf':[1,3,5],'min_samples_split':[2,4,6]}
#gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=5).fit(X,A)
#gscv.best_params_

#output: {'learning_rate': 0.01,
# 'max_depth': 2,
# 'min_samples_leaf': 1,
# 'min_samples_split': 2,
# 'n_estimators': 150}

In [11]:
#high dim grid search (commented out since it takes a few minutes to run)

#X=highDim_dataset.iloc[:,2:].values
#A=highDim_dataset['A'].values
#Y=highDim_dataset['Y'].values

#params = {'learning_rate':[0.01,0.05,0.1,0.5], 'max_depth': [1,2,3,4], 'n_estimators':[50,100,150],
#          'min_samples_leaf':[1,3,5],'min_samples_split':[2,4,6]}
#gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=5).fit(X,A)
#gscv.best_params_


#output: {'learning_rate': 0.05,
# 'max_depth': 1,
# 'min_samples_leaf': 5,
# 'min_samples_split': 2,
# 'n_estimators': 100}

## Write Propensity Scores to CSV

In [12]:
X=lowDim_dataset.iloc[:,2:].values
A=lowDim_dataset['A'].values
Y=lowDim_dataset['Y'].values

gbm = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 2, min_samples_leaf = 1,
                                min_samples_split = 2, n_estimators = 150).fit(X,A)

low_dim_propensity_scores = [x[1] for x in gbm.predict_proba(X)]

In [13]:
X=highDim_dataset.iloc[:,2:].values
A=highDim_dataset['A'].values
Y=highDim_dataset['Y'].values

gbm = GradientBoostingClassifier(learning_rate = 0.05, max_depth = 1, min_samples_leaf = 5,
                                min_samples_split = 2, n_estimators = 100).fit(X,A)

high_dim_propensity_scores = [x[1] for x in gbm.predict_proba(X)]

In [14]:
pd.DataFrame({'propensity_scores':low_dim_propensity_scores}).to_csv('../output/low_dim_propensity_scores.csv')
pd.DataFrame({'propensity_scores':high_dim_propensity_scores}).to_csv('../output/high_dim_propensity_scores.csv')

## Matching

In [15]:
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)
packnames = ('optmatch')

In [16]:
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

In [17]:
optmatch = rpackages.importr('optmatch')

In [18]:
with localconverter(robjects.default_converter + pandas2ri.converter):
    lowDim_dataset_R = robjects.conversion.py2rpy(lowDim_dataset)
    lowDim_dataset_propensity_R = robjects.conversion.py2rpy(lowDim_dataset_propensity)
    lowDim_dataset_linear_propensity_R = robjects.conversion.py2rpy(lowDim_dataset_linear_propensity)

In [19]:
robjects.r.head(lowDim_dataset_propensity_R)

Y,A,V1,...,V21,V22,propensity_score
19.678858,0,1.590000,...,0.000000,1.309683,0.405085
17.842989,0,0.000000,,0.000000,1.719547,0.322207
22.108788,1,0.000000,,2.120000,0.996210,0.298967
15.355899,0,0.000000,,0.000000,1.504077,0.175873
16.787813,1,1.810000,,0.000000,0.327864,0.159559
11.378754,0,0.000000,,0.000000,0.405465,0.146625


### Method 1: Mahalanobis

### Method 2: Propensity Score

In [20]:
full_match_propensity_factor = optmatch.fullmatch(optmatch.match_on(Formula('A~propensity_score'),data=lowDim_dataset_propensity_R,method='euclidean'),data=lowDim_dataset_propensity_R)

After the above step, we can do the rest of the code in the implement_full_match.R using python functions.

In [21]:
lowDim_dataset_propensity['assign'] = list(full_match_propensity_factor)

In [22]:
#example group
lowDim_dataset_propensity.loc[lowDim_dataset_propensity['assign']==67][['Y','A','assign','propensity_score']]

,Y,A,assign,propensity_score
185,18.392843,0,67,0.292125
407,23.001812,1,67,0.293804


In [23]:
#if this doesn't print anything then that means each group has at least one control and at least one treatment which is good

for i in range(max(list(full_match_propensity_factor))):
    temp = lowDim_dataset_propensity.loc[lowDim_dataset_propensity['assign']==i+1][['Y','A','assign','propensity_score']]
    grouping = temp['A'].values
    
    if (sum(grouping)==0 or sum(grouping)==len(grouping)):
        print(i+1)    

In [24]:
#compute ATE
ATE_vec = []
weights = []

for i in range(max(list(full_match_propensity_factor))):
    temp = lowDim_dataset_propensity.loc[lowDim_dataset_propensity['assign']==i+1]
    
    treatment_Y = temp.loc[temp['A']==1]['Y'].values
    control_Y = temp.loc[temp['A']==0]['Y'].values
    
    ATE_vec.append(np.mean(treatment_Y)-np.mean(control_Y))
    weights.append(len(treatment_Y)+len(control_Y))
    

In [25]:
propensity_est_ATE=np.average(ATE_vec, weights=weights)
propensity_est_ATE

3.387793812531444